In [100]:
import numpy as np
from PIL import Image as im
import os
import matplotlib.pyplot as plt
from scipy.stats import mode
from skimage.morphology import binary_dilation, binary_erosion, binary_closing
import skimage
from skimage import io



# Turning euler angle RGB values to an Image


In [101]:
# Here we open up our csv file and asign it to file
file = open("11CSR01-p Specimen 1 Area 2 Montaged Data 1 Montaged Map Data-Ph + AE + BC + EDS (Al+Ca+Na+Fe+Si+K) (1).csv")

In [102]:
#Checks to see if a directory called "Euler_Images" exists, if not it creates one
if not os.path.exists("Euler_Images"):
    os.makedirs("Euler_Images")

#checks to see if a directory called "Chemistry_Images" exists, if not it creates one
if not os.path.exists("Chemistry_Images"):
    os.makedirs("Chemistry_Images")
    

In [103]:
#This code is based on a function developed by previous semester's group and professor Matty's script
#This functions reads through a csv file which is contain in this folder and thens creats an RGBA image based on the euler
#and rgb value assigned to it by a previous program
#can take a very long time with the intended file so be prepared to wait!
def get_phase_color(file, save_image=True):
    # skip the first two lines because thats just the header
    file.seek(0)
    file.readline()
    file.readline()

    # find the max x and y values to determine the size of the image
    max_x = 0
    max_y = 0
    for line in file:
        line = line.split(",")
        x = int(line[1])//10
        y = int(line[2])//10

        if (x > max_x):
            max_x = x
        if (y > max_y):
            max_y = y
    print(max_x, max_y)
    width =  max_x + 1
    height = max_y + 1
    file.seek(0)
    file.readline()
    file.readline()
    
    # create a 3d array of zeros with the size of the image
    euler_phase = np.zeros((height, width, 4))
    #read through each line of the data file and assign the correspond x,y values pixel its rgba value
    for line in file:
        line = line.split(",")
        rgb = line[4].split() #line[4] is the rgb value
        y = int(line[1])//10 #line[1] is the x value
        x = int(line[2])//10 #line[2] is the y value

        rgb = np.array(rgb, "int")
        #if line[3] is 1 then the pixel is a grain and we assign it the rgb value
        if (int(line[3]) == 1):
            rgb = rgb/255
            #add 0.5 alpha value to rgb
            rgba = np.append(rgb,0.5)
            euler_phase[x,y,:] = rgba
            
        else:
            euler_phase[x,y,:] = [1,1,1,0]

    #save the image as a png file to the Euler_Images folder
    if save_image:
        plt.imsave("Euler_Images/euler_phase.png", euler_phase)

    return euler_phase

#note that input file columns must match and be the same as the csv file included in this Folder
euler_phase = get_phase_color(file)

3522 2027


# Turning the chemical values into images

In [ ]:
#This function takes in a csv file to then read through it and create an image based on the chemical value
#the chemical value is normalized by the max value of that chemical in the csv file

#inputs are a csv file, a list of the max chemical values, and the chemical you want to create an image of
#note the input for chemical is the index of the chemical in the csv file in the cells its from
# based off of the order (Al, Ca, Na, Fe, Si, K)
def get_chem(file,max_chemicals, save_image=True,chemical=1):
    # skip the first two lines because thats just the header
    file.seek(0)
    file.readline()
    file.readline()

    # find the max x and y values to determine the size of the image
    max_x = 0
    max_y = 0

    for line in file:
        line = line.split(",")
        x = int(line[1])//10
        y = int(line[2])//10

        if (x > max_x):
            max_x = x
        if (y > max_y):
            max_y = y

    width =  max_x + 1
    height = max_y + 1
    file.seek(0)
    file.readline()
    file.readline()
    
    chemical_img = np.zeros((height, width))

    #read through each line of the data file and assign the correspond x,y values pixel its chemical value
    for line in file:
        line = line.split(",")
        y = int(line[1])//10
        x = int(line[2])//10

        # if (int(line[3]) == 1):
        chemical_values = line[6].split(' ')
        #normalize the chemical values
        chemical_values = np.array(chemical_values, "float")
        chemical_values = chemical_values/max_chemicals[chemical]
            
        chemical_img[x,y] = chemical_values[chemical]


 
    print("Done with Chemical: ",chemical)
    return chemical_img



In [ ]:
#function to find the max chemical value in the csv file
#each chemicals max value is returned in a list 
# the order of the chemicals should be (Al, Ca, Na, Fe, Si, K)
def find_max_of_chem(file):
    file.seek(0)
    file.readline()
    file.readline()

    max_chem_0 = 0
    max_chem_1 = 0
    max_chem_2 = 0
    max_chem_3 = 0
    max_chem_4 = 0
    max_chem_5 = 0

    for line in file:
        line = line.split(",")



        chemical_values = line[6].split(' ')
        chemical_values = np.array(chemical_values, "float")
        if (chemical_values[0] > max_chem_0):
            max_chem_0 = chemical_values[0]
        if (chemical_values[1] > max_chem_1):
            max_chem_1 = chemical_values[1]
        if (chemical_values[2] > max_chem_2):
            max_chem_2 = chemical_values[2]
        if (chemical_values[3] > max_chem_3):
            max_chem_3 = chemical_values[3]
        if (chemical_values[4] > max_chem_4):
            max_chem_4 = chemical_values[4]
        if (chemical_values[5] > max_chem_5):
            max_chem_5 = chemical_values[5]
            
        
    return [max_chem_0,max_chem_1,max_chem_2,max_chem_3,max_chem_4,max_chem_5]  

 


In [ ]:
max_chemicals = find_max_of_chem(file)
print(max_chemicals)

[69401.7, 37805.1, 32040.8, 19595.0, 159324.4, 33571.3]


In [ ]:

AL_img = get_chem(file,max_chemicals,chemical=0)
CA_img = get_chem(file,max_chemicals,chemical=1)
NA_img = get_chem(file,max_chemicals,chemical=2)
FE_img = get_chem(file,max_chemicals,chemical=3)
SI_img = get_chem(file,max_chemicals,chemical=4)
K_img = get_chem(file,max_chemicals,chemical=5)





Done with Chemical:  0
Done with Chemical:  1
Done with Chemical:  2
Done with Chemical:  3
Done with Chemical:  4
Done with Chemical:  5


In [ ]:
#function takes in a image, creates a mask, finds the area of differnt regions,
#and then removes regions that are below a certain area depending on the input
def reduce_area(image,area):
    labeled = skimage.morphology.label(image)
    my_regions = skimage.measure.regionprops_table(labeled, properties=('label','area'))
    for i in range(len(my_regions['label'])):
        if my_regions['area'][i] < area:
            labeled[labeled == my_regions['label'][i]] = 0
    
    return labeled


In [ ]:
from scipy.ndimage import generic_filter
#This cell defines a method to use a sliding wndow that slides a 3x3 window across the image and sets the center pixel to the max value of the window
#The main function is called my_max_neighbor
#Function can take a while especially with large images
# def max_neighbor_chem(window):
#     flat = window.flatten()
#     flat = flat.tolist()
#     flat = mode(flat)
#     return flat[0]

# def my_max_neighbor_chem(image):
#     half_window = 1
#     padded_image = np.pad(image, pad_width=(half_window, half_window), mode='constant', constant_values=0)

#     for i in range(half_window, padded_image.shape[0] - half_window):
#         for j in range(half_window, padded_image.shape[1] - half_window):
#             window = padded_image[i-half_window:i+half_window+1, j-half_window:j+half_window+1]
#             padded_image[i, j] = max_neighbor_chem(window)

#     # get rid of the padding
#     unpadded = padded_image[half_window:-half_window, half_window:-half_window]

#     return unpadded


def max_neighbor_chem(window):
    return mode(window)[0]

def my_max_neighbor_chem(image):
    half_window = 1
    size = 2*half_window + 1
    result = generic_filter(image, max_neighbor_chem, size=size)
    return result

from skimage.filters.rank import modal
from skimage.morphology import square

def my_modal_filter(image):
    half_window = 1
    size = 2*half_window + 1
    result = modal(image, square(size))
    return result


In [ ]:
# Define your directory path
directory_path = 'Chemistry_images/reduced/v2/'

# Create the directory if it does not exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)


Chemistry_directory_reduced = "Chemistry_images/reduced/v2/"
copy_SI_img = SI_img.copy()
copy_SI_img[copy_SI_img < 0.5] = 0
copy_SI_img[copy_SI_img > 0.5] = 1

copy_AL_img = AL_img.copy()
copy_AL_img[copy_AL_img < 0.2] = 0
copy_AL_img[copy_AL_img > 0.2] = 1

copy_CA_img = CA_img.copy()
copy_CA_img[copy_CA_img < 0.2] = 0
copy_CA_img[copy_CA_img > 0.2] = 1

copy_FE_img = FE_img.copy()
copy_FE_img[copy_FE_img < 0.2] = 0
copy_FE_img[copy_FE_img > 0.2] = 1

copy_K_img = K_img.copy()
copy_K_img[copy_K_img < 0.2] = 0
copy_K_img[copy_K_img > 0.2] = 1

copy_NA_img = NA_img.copy()
copy_NA_img[copy_NA_img < 0.2] = 0
copy_NA_img[copy_NA_img > 0.2] = 1






max_AL_img = my_modal_filter(copy_AL_img)
max_AL_img = reduce_area(max_AL_img,100)
AL_img = im.fromarray((max_AL_img ).astype(np.uint8), mode="L")
AL_img.save(Chemistry_directory_reduced+"AL_img.png")
print("Done with AL")

max_CA_img = my_modal_filter(copy_CA_img )
# max_CA_img = reduce_area(max_CA_img,100)
CA_img = im.fromarray((max_CA_img ).astype(np.uint8), mode="L")
CA_img.save(Chemistry_directory_reduced+"CA_img.png")
print("Done with CA")

max_FE_img = my_modal_filter(copy_FE_img )
max_FE_img = reduce_area(max_FE_img,100)
FE_img = im.fromarray((max_FE_img ).astype(np.uint8), mode="L")
FE_img.save(Chemistry_directory_reduced+"FE_img.png")
print("Done with FE")

max_K_img = my_modal_filter(copy_K_img)
max_K_img = reduce_area(max_K_img,100)
K_img = im.fromarray((max_K_img ).astype(np.uint8), mode="L")
K_img.save(Chemistry_directory_reduced+"K_img.png")
print("Done with K")

max_NA_img = my_modal_filter(copy_NA_img)
max_NA_img = reduce_area(max_NA_img,100)
NA_img = im.fromarray((max_NA_img ).astype(np.uint8), mode="L")
NA_img.save(Chemistry_directory_reduced+"NA_img.png")
print("Done with NA")

max_SI_img = my_modal_filter(copy_SI_img)
max_SI_img = reduce_area(max_SI_img,100)
SI_img = im.fromarray((max_SI_img ).astype(np.uint8), mode="L")
SI_img.save(Chemistry_directory_reduced+"SI_img.png")
print("Done with SI")

#save the cropeed image
SI_image = im.fromarray((max_SI_img * 255).astype(np.uint8), mode="L")
SI_image.save(Chemistry_directory_reduced+"SI_img3.png")

SI_image = im.fromarray((copy_SI_img * 255).astype(np.uint8), mode="L")
SI_image.save(Chemistry_directory_reduced+"SI_img2.png")


TypeError: '<' not supported between instances of 'Image' and 'float'

In [ ]:
# SI_image3 = im.open(Chemistry_directory+"SI_img3.png")
# SI_image3 = np.array(SI_image3)

# SI_image4 = reduce_area(SI_image3,100)
# SI_image4 = im.fromarray((SI_image4 * 255).astype(np.uint8), mode="L")
# SI_image4.save(Chemistry_directory+"SI_img4.0.png")

In [ ]:
Chemistry_directory="Chemistry_images/"

# Lets overlay the Feltz image with the band contrast image of SI just to see what it looks like

In [13]:
########################################################

# THIS CELLS CODE IS FROM JOELS BOUNDERIES.IPYNB FILE   #

########################################################
from skimage.filters import sobel
from skimage.segmentation import felzenszwalb
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage.measure import label
from skimage.color import label2rgb
from skimage import color
def preprocess(image):
    image = img_as_float(image)
    if image.shape[2] > 3:
        image = image[:,:,:3]
    gimage = color.rgb2gray(image)
    return skimage.exposure.equalize_hist(gimage)

def get_felzenszwalb(image, scale=1.0, sigma=0.8, min_size=10):
    '''
        Performs Felzenszwalbs segmentation and returns the segmented mask
    '''
    return felzenszwalb(image, scale=scale, sigma=sigma, min_size=min_size)

def get_labeled_and_overlayed(image, mask):
    '''
        Generates the labeled image of `image`, overlays the labeled image it
        on top of `image`, and returns both the labeled and overlayed images
    '''
    labeled = label(mask)
    overlayed = label2rgb(labeled, image=image)
    return labeled, overlayed

def get_marked_boundaries(image, mask, color='red', background=True):
    colors = {'red':(1,0,0), 'green':(0,1,0), 'blue':(0,0,1), 'yellow':(1,1,0), 'white':(1,1,1) }
    assert(colors[color] is not None)

    if not background:
        image = np.zeros_like(image)

    # Cast to [0..255] so it can be saved
    return (mark_boundaries(image, mask, color=colors[color]) * 255).astype(np.uint8)

def outline_boundaries(image, marked, color='black'):
    '''
        Returns a copy of `image` overlayed with `marked`
    '''
    print(image.shape, image.dtype, marked.shape, marked.dtype)
    assert(image.shape == marked.shape)

    colors = {'red':0, 'green':1, 'blue':2, 'black':3, 'white':4, 'yellow':5}
    assert(colors[color] is not None)

    outlined = image.copy()
    for i in range(marked.shape[0]):
        for j in range(marked.shape[1]):
            if marked[i,j,:].any() > 0:
                if color == 'black':
                    outlined[i,j,:] = 0
                elif color == 'white':
                    outlined[i,j,:] = 1
                elif color == 'yellow':
                    outlined[i,j,0:2] = 1
                else:
                    outlined[i,j,v] = 1
    return (outlined * 255).astype(np.uint8)


def segment_boundaries(image, scale, sigma, min_size, outline_color='black'):
    '''
        Segments the boundaries of an image using Felzenszwalb, marks the boundaries
        and returns the boundaries segmented onto the labeled image
    '''
    # Get the segmentation mask using Felzenszwalb
    mask = get_felzenszwalb(image, scale=scale, sigma=sigma, min_size=min_size)
    # Mark boundaries of the image using the mask
    marked = get_marked_boundaries(image, mask, background=False)
    # Get the labeled image overlayed with the mask
    _, overlay = get_labeled_and_overlayed(image, mask)
    # Return the segmented, labeled overlayed image with its boundaries outlined
    return outline_boundaries(overlay, marked, outline_color)


# filename = 'Chemistry_images/SI_img4.png'
# image = io.imread(filename)
# segmented = segment_boundaries(image, scale=300, sigma=0.8, min_size=200, outline_color='black')
# plt.imshow(segmented)

In [14]:
def overlay_feltz(band_con, feltz):
    for x in range(feltz.shape[0]):
        for y in range(feltz.shape[1]):
            if (feltz[x,y].all() > 0):
                band_con[x,y] = feltz[x,y]
    return band_con


In [44]:


feltz= im.open("felz_SI.png")
band = im.open("band_contrast.png")
feltz = np.array(feltz)
band = np.array(band)

#lets change band to rgb
band = skimage.color.gray2rgb(band)


overlayed = overlay_feltz(band, feltz)

overlayed = im.fromarray((overlayed ).astype(np.uint8), mode="RGB")
overlayed.save("overlayed.png")


In [ ]:


#create the chemistry directory if it doestn already exist
if not os.path.exists(Chemistry_directory):
    os.makedirs(Chemistry_directory)


# AL_image = im.fromarray((AL_img * 255).astype(np.uint8), mode="L")
# AL_image.save(Chemistry_directory+"AL_img.png")
# CA_image = im.fromarray((CA_img * 255).astype(np.uint8), mode="L")
# CA_image.save(Chemistry_directory+"CA_img.png")
# NA_image = im.fromarray((NA_img * 255).astype(np.uint8), mode="L")
# NA_image.save(Chemistry_directory+"NA_img.png")
# FE_image = im.fromarray((FE_img * 255).astype(np.uint8), mode="L")
# FE_image.save(Chemistry_directory+"FE_img.png")
SI_image = im.fromarray((SI_img * 255).astype(np.uint8), mode="L")
SI_image.save(Chemistry_directory+"SI_img.png")
# K_image = im.fromarray((K_img * 255).astype(np.uint8), mode="L")
# K_image.save(Chemistry_directory+"K_img.png")

# Next step is to be able to quanatize our euler phase image and be able to change it to create a viable mask we 
# can use to find quartz :)

In [6]:

from skimage import io
import matplotlib.pyplot as plt
from skimage import morphology
from skimage import measure
import numpy as np
from skimage import color
import skimage as skimage
import scipy.ndimage as ndi
import numpy as np
from skimage import color, filters
from scipy.stats import mode
import imageio

In [60]:
image = io.imread('/Users/oasis/Desktop/GeoProj/11-3-23_From Matty/Euler_images/euler_phase_to_color.png')
newImage = image
if newImage.shape[2] == 4:
    newImage = newImage[:, :, :3]


In [61]:
def quantization(img, L):
    '''
    Quantize the image img (with 256 gray levels) to L gray levels (in the example above, 
    image was quantized to 4 gray levels)
    Return the quantized image
    '''
    factor = 256/L
    img2 = img // factor
    return img2 * factor

In [62]:
quna_Eul_img_16 = quantization(newImage,16)
# quna_Eul_img_32 = quantization(newImage,32)
# quna_Eul_img_64 = quantization(newImage,64)
# quna_Eul_img_128 = quantization(newImage,128)


imageio.imsave('Euler_Images/quan_image_colored_16.png', quna_Eul_img_16.astype('uint8'))# save image
# imageio.imsave('quan_image_colored_32.png', quna_Eul_img_32.astype('uint8') * 255)# save image
# imageio.imsave('quan_image_colored_64.png', quna_Eul_img_64.astype('uint8') * 255)# save image
# imageio.imsave('quan_image_colored_128.png', quna_Eul_img_128.astype('uint8') * 255)# save image

In [63]:
import numpy as np
from scipy import ndimage
from scipy.stats import mode

In [69]:
def max_red(window):
    #go through the window of pixels and find the red value that appears the most
    red = window[:,:,0]
    red = red.flatten()
    red = red.tolist()
    red = mode(red)
    return red[0]

def max_green(window):
   #go through the window of pixels and find the gree value that appears the most
    green = window[:,:,1]
    green = green.flatten()
    green = green.tolist()
    green = mode(green)
    return green[0]
def max_blue(window):
    #go through the window of pixels and find the blue value that appears the most
    blue = window[:,:,2]
    blue = blue.flatten()
    blue = blue.tolist()
    blue = mode(blue)
    return blue[0]

def max_neighbor(window):
    max_B = max_blue(window)
    max_R = max_red(window)
    max_G = max_green(window)
    return np.array([max_R, max_G, max_B])

def my_max_neighbor(image):
    half_window = 1
    # pad the image to avoid errors during edge cases
    padded_image = np.pad(image, pad_width=((half_window, half_window), (half_window, half_window), (0, 0)), mode='constant', constant_values=0)

    # loop through the image with a sliding window
    for i in range(half_window, padded_image.shape[0] - half_window):
        for j in range(half_window, padded_image.shape[1] - half_window):
            window = padded_image[i-half_window:i+half_window+1, j-half_window:j+half_window+1, :]
            padded_image[i, j] = max_neighbor(window)

    # get rid of the padding
    unpadded = padded_image[half_window:-half_window, half_window:-half_window]

    return unpadded

In [82]:
from scipy.ndimage import maximum_filter

def my_max_neighbor_fast(image, channel):
    half_window = 1
    footprint = np.ones((2*half_window+1, 2*half_window+1))
    result = maximum_filter(image[:,:,channel], footprint=footprint)
    return result

In [84]:
# Note be prepared this can take a long time to run for each one thats why initially i have only one uncommented
# Running on a m2 mac book pro takes around 20-30 minutes for just max_16
from datetime import datetime


max_16 = quna_Eul_img_16
print(max_16.shape)
red_channel = my_max_neighbor_fast(quna_Eul_img_16, 0)
green_channel = my_max_neighbor_fast(quna_Eul_img_16, 1)
blue_channel = my_max_neighbor_fast(quna_Eul_img_16, 2)
max_16 = np.stack((red_channel, green_channel, blue_channel), axis=-1)
imageio.imsave('Euler_Images/max_16_vec.png', max_16.astype('uint8') )# save image
print("Current execution time: ", datetime.now())

# max_16 = my_max_neighbor(quna_Eul_img_16)
# imageio.imsave('Euler_Images/max_16.png', max_16.astype('uint8') * 255)# save image
# print("Current execution time: ", datetime.now())
# max_32 = my_max_neighbor(quna_Eul_img_32)
# imageio.imsave('max_32.png', max_32.astype('uint8') * 255)# save image
# max_64 = my_max_neighbor(quna_Eul_img_64)
# imageio.imsave('max_64.png', max_64.astype('uint8') * 255)# save image
# max_128 = my_max_neighbor(quna_Eul_img_128)
# imageio.imsave('max_128.png', max_128.astype('uint8') * 255)# save image
#imageio.imsave('max_16.png', max_16.astype('uint8') * 255)# save image
# max_32 = my_max_neighbor(quna_Eul_img_32)
# imageio.imsave('max_32.png', max_32.astype('uint8') * 255)# save image
# max_64 = my_max_neighbor(quna_Eul_img_64)
# imageio.imsave('max_64.png', max_64.astype('uint8') * 255)# save image
# max_128 = my_max_neighbor(quna_Eul_img_128)
# imageio.imsave('max_128.png', max_128.astype('uint8') * 255)# save image

(2028, 3523, 3)
Current execution time:  2023-12-13 14:26:20.402739


In [69]:
#label max_16 then reduce where the area is less and area
def reduce_area(image,area):
    labeled = skimage.morphology.label(image)
    my_regions = skimage.measure.regionprops_table(labeled, properties=('label','area'))

    for i in range(len(my_regions['label'])):
        if my_regions['area'][i] < area:
            labeled[labeled == my_regions['label'][i]] = 0
    
    return labeled

In [75]:
max_16 =io.imread('Euler_Images/max_16.png')
grayscale = color.rgb2gray(max_16)
thresh = skimage.filters.threshold_otsu(grayscale)
binary = grayscale > thresh

reduced_Binary = reduce_area(binary,100)


In [76]:
reduced_Binary = reduce_area(binary,100)


In [77]:
#whereever reduced_Binaray is 0, make max_16_reduced 0
max_16_reduced = max_16
for i in range(len(reduced_Binary)):
    for j in range(len(reduced_Binary[0])):
        if reduced_Binary[i][j] == 0:
            max_16_reduced[i][j] = 0

imageio.imsave('Euler_Images/max_16_reduced.png', max_16_reduced.astype('uint8') * 255)

# Now that we have a mask for euler angles and a way to make chemistry masks next step is to make them work together
- Quartz chrystal needs silicon so if there is no silicon where that euler angle is kick it out 

In [56]:
#intersect_of_two calculates the intersection between two masks so that mask1
# must contain mask2 or the regions that dont intersect with a region in mask2 
# will be set to 0
def labels_intersect(labeled1, label1, labeled2, label2):
    return np.any(np.logical_and(labeled1 == label1, labeled2 == label2))

def intersect_of_two(mask1,mask2):
    labeled1 = skimage.morphology.label(mask1)
    labeled2 = skimage.morphology.label(mask2)
  
    for label1 in np.unique(labeled1):
        intersects = False
        for label2 in np.unique(labeled2):
            if labels_intersect(labeled1, label1, labeled2, label2):
                intersects = True
                break
        if not intersects:
            labeled1[labeled1 == label1] = 0

    return labeled1



#match keep is a function that takes in two masks the idea is that quartz must contain silicon so 
#mask 1 is euler mask 2 is SI mas. THe function will go through the eulers mask and the SI mask 
# has a region within the same region in the euler mask then it will keep it otherwise it will set it to 0
#ultimately we want to return the euler mask that is either reduced or the same depending 
# on the outcome of the operations
#


In [58]:
Euler_mask = io.imread('Euler_Images/max_16_reduced.png')
SI_mask = io.imread('Chemistry_images/SI_img4.png')
SI_mask = skimage.color.gray2rgb(SI_mask)

Euler_mask = Euler_mask[:1000,:1000]
SI_mask = SI_mask[:1000,:1000]

imageio.imsave('Euler_Images/Euler_thou.png', Euler_mask.astype('uint8') )

matched_mask = intersect_of_two(Euler_mask,SI_mask)
Euler_mask[matched_mask == 0] = 0
imageio.imsave('Euler_Images/matched_Euler.png', Euler_mask.astype('uint8') )




In [9]:
#lets load the reduced cheistry images

AL_img = io.imread('Chemistry_images/reduced/AL_img.png')
CA_img = io.imread('Chemistry_images/reduced/CA_img.png')
NA_img = io.imread('Chemistry_images/reduced/NA_img.png')
FE_img = io.imread('Chemistry_images/reduced/FE_img.png')
SI_img = io.imread('Chemistry_images/reduced/SI_img.png')
K_img = io.imread('Chemistry_images/reduced/K_img.png')

#turn images to binary images

thresh = skimage.filters.threshold_otsu(AL_img)
AL_img = AL_img > thresh

thresh = skimage.filters.threshold_otsu(CA_img)
CA_img = CA_img > thresh


thresh = skimage.filters.threshold_otsu(NA_img)
NA_img = NA_img > thresh


thresh = skimage.filters.threshold_otsu(FE_img)
FE_img = FE_img > thresh

thresh = skimage.filters.threshold_otsu(SI_img)
SI_img = SI_img > thresh


thresh = skimage.filters.threshold_otsu(K_img)
K_img = K_img > thresh

#now give lets change it so instead of binary these images are going to be turned into RGB images but asign
# a color to each chemical

def colorize_non_black_pixels(img, color):
    rgb_img = skimage.color.gray2rgb(img)
    for i in range(3):
        rgb_img[:,:,i] = np.where(img != 0, color[i], 0)
    return rgb_img

AL_img = colorize_non_black_pixels(AL_img, [1, 0, 0])
CA_img = colorize_non_black_pixels(CA_img, [0, 1, 0])
NA_img = colorize_non_black_pixels(NA_img, [0, 0, 1])
FE_img = colorize_non_black_pixels(FE_img, [0, 1, 1])
SI_img = colorize_non_black_pixels(SI_img, [1, 0, 1])
K_img = colorize_non_black_pixels(K_img, [1, 1, 0])

#now lets store these in a file called Chemistry_images/RGB
#first lets make sure the directory exists
if not os.path.exists("Chemistry_images/RGB"):
    os.makedirs("Chemistry_images/RGB")
imageio.imsave('Chemistry_images/RGB/AL_img.png', AL_img.astype('uint8') * 255)# save image
imageio.imsave('Chemistry_images/RGB/CA_img.png', CA_img.astype('uint8') * 255)# save image
imageio.imsave('Chemistry_images/RGB/NA_img.png', NA_img.astype('uint8') * 255)# save image
imageio.imsave('Chemistry_images/RGB/FE_img.png', FE_img.astype('uint8') * 255)# save image
imageio.imsave('Chemistry_images/RGB/SI_img.png', SI_img.astype('uint8') * 255)# save image
imageio.imsave('Chemistry_images/RGB/K_img.png', K_img.astype('uint8') * 255)# save image




In [10]:
#check to see if directery Chemistry_images/overlayed exists if not create it
if not os.path.exists("Chemistry_images/overlayed"):
    os.makedirs("Chemistry_images/overlayed")

#load band_contrast.png 
band = io.imread('band_contrast.png')

In [28]:
#load the RGB images as numpy arrays
AL_img = io.imread('Chemistry_images/RGB/AL_img.png')
CA_img = io.imread('Chemistry_images/RGB/CA_img.png')
NA_img = io.imread('Chemistry_images/RGB/NA_img.png')
FE_img = io.imread('Chemistry_images/RGB/FE_img.png')
SI_img = io.imread('Chemistry_images/RGB/SI_img.png')
K_img = io.imread('Chemistry_images/RGB/K_img.png')

#need to turn this image from RGB into an RGBA image but all pixels that are black need an alpha channel value of 0

def RGB_2_RGBA(img):
    rgba_img = np.dstack((img, np.ones((img.shape[0], img.shape[1]))*255)) # Add alpha channel with value 255 (opaque)
    rgba_img[(rgba_img[:,:,0] == 0) & (rgba_img[:,:,1] == 0) & (rgba_img[:,:,2] == 0), 3] = 0 # Set alpha to 0 for black pixels
    return rgba_img

AL_img = RGB_2_RGBA(AL_img)
CA_img = RGB_2_RGBA(CA_img)
NA_img = RGB_2_RGBA(NA_img)
FE_img = RGB_2_RGBA(FE_img)
SI_img = RGB_2_RGBA(SI_img)
K_img = RGB_2_RGBA(K_img)

#if we dont have a directory call chemistryimage/rgba create it
if not os.path.exists("Chemistry_images/RGBA"):
    os.makedirs("Chemistry_images/RGBA")

#save the images as png files
imageio.imsave('Chemistry_images/RGBA/AL_img.png', AL_img.astype('uint8') )# save image
imageio.imsave('Chemistry_images/RGBA/CA_img.png', CA_img.astype('uint8'))# save image
imageio.imsave('Chemistry_images/RGBA/NA_img.png', NA_img.astype('uint8') )# save image
imageio.imsave('Chemistry_images/RGBA/FE_img.png', FE_img.astype('uint8'))# save image
imageio.imsave('Chemistry_images/RGBA/SI_img.png', SI_img.astype('uint8') )# save image
imageio.imsave('Chemistry_images/RGBA/K_img.png', K_img.astype('uint8') )# save image



In [40]:
#now lets overlay the band_contrast.png with the RGBA images

AL_img = io.imread('Chemistry_images/RGBA/AL_img.png')
CA_img = io.imread('Chemistry_images/RGBA/CA_img.png')
NA_img = io.imread('Chemistry_images/RGBA/NA_img.png')
FE_img = io.imread('Chemistry_images/RGBA/FE_img.png')
SI_img = io.imread('Chemistry_images/RGBA/SI_img.png')
K_img = io.imread('Chemistry_images/RGBA/K_img.png')

band = io.imread('band_contrast.png')
band = skimage.color.gray2rgb(band)
band = np.dstack((band, np.ones((band.shape[0], band.shape[1]))*255)) 
print(band.shape)
#lets make sure the directory exists
if not os.path.exists("Chemistry_images/overlayed"):
    os.makedirs("Chemistry_images/overlayed")

#lets make a new fucntion for overlaying the band_contrast.png with the RGBA images

def overlay_band(band, img):
    band = band.copy()
    #where ever the img alpha value is not 0 make the band value equal to the img value
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if (img[x,y,3] != 0):
                band[x,y] = img[x,y]
    return band

 


#lets overlay the band_contrast.png with the RGBA images
overlayed_AL = overlay_band(band, AL_img)
overlayed_CA = overlay_band(band, CA_img)
overlayed_NA = overlay_band(band, NA_img)
overlayed_FE = overlay_band(band, FE_img)
overlayed_SI = overlay_band(band, SI_img)
overlayed_K = overlay_band(band, K_img)

#lets save the overlayed images
imageio.imsave('Chemistry_images/overlayed/overlayed_AL.png', overlayed_AL.astype('uint8') )# save image
imageio.imsave('Chemistry_images/overlayed/overlayed_CA.png', overlayed_CA.astype('uint8') )# save image
imageio.imsave('Chemistry_images/overlayed/overlayed_NA.png', overlayed_NA.astype('uint8') )# save image
imageio.imsave('Chemistry_images/overlayed/overlayed_FE.png', overlayed_FE.astype('uint8') )# save image
imageio.imsave('Chemistry_images/overlayed/overlayed_SI.png', overlayed_SI.astype('uint8') )# save image
imageio.imsave('Chemistry_images/overlayed/overlayed_K.png', overlayed_K.astype('uint8') )# save image


(2028, 3523, 4)


# Change reduced chemistry images to binary and XOR them with silicon turning all pixels we definetly dont want 
# to look for off and creating a mask of regions we forsure want to look at

In [42]:
AL_img = io.imread('Chemistry_images/reduced/AL_img.png')
CA_img = io.imread('Chemistry_images/reduced/CA_img.png')
NA_img = io.imread('Chemistry_images/reduced/NA_img.png')
FE_img = io.imread('Chemistry_images/reduced/FE_img.png')
SI_img = io.imread('Chemistry_images/reduced/SI_img.png')
K_img = io.imread('Chemistry_images/reduced/K_img.png')

#turn images to binary images

thresh = skimage.filters.threshold_otsu(AL_img)
AL_img = AL_img > thresh

thresh = skimage.filters.threshold_otsu(CA_img)
CA_img = CA_img > thresh


thresh = skimage.filters.threshold_otsu(NA_img)
NA_img = NA_img > thresh


thresh = skimage.filters.threshold_otsu(FE_img)
FE_img = FE_img > thresh

thresh = skimage.filters.threshold_otsu(SI_img)
SI_img = SI_img > thresh


thresh = skimage.filters.threshold_otsu(K_img)
K_img = K_img > thresh

#now the final image will be the result of XOR'ing all the binary images together

def xor_image_withSI (SI, img):
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if (SI[x,y] == 1) and (img[x,y] == 1):
                SI[x,y] = 0
                
    return SI

xor_SI = xor_image_withSI(SI_img, AL_img)
xor_SI = xor_image_withSI(xor_SI, CA_img)
xor_SI = xor_image_withSI(xor_SI, NA_img)
xor_SI = xor_image_withSI(xor_SI, FE_img)
xor_SI = xor_image_withSI(xor_SI, K_img)

#now save he xor_SI image
imageio.imsave('Chemistry_images/xor_SI.png', xor_SI.astype('uint8') * 255)# save image



# Now using the chemistry mask we can further reduce areas of interest and finer pinpoint regions we desire
# by using a binary fersion of the euler redeced mask and using AND operations to keep what we want 

In [56]:
#load max_16_reduced.png
max_16_reduced = io.imread('Euler_Images/max_16_reduced.png')
xor_SI = io.imread('Chemistry_images/xor_SI.png')
#lets turn max_16_reduced into a binary image

max_16_reduced = skimage.color.rgb2gray(max_16_reduced)
thresh = skimage.filters.threshold_otsu(max_16_reduced)
max_16_reduced[max_16_reduced < thresh] = 0
max_16_reduced[max_16_reduced > thresh] = 1


#save the binary image in a folder called Euler_Images/binary
if not os.path.exists("Euler_Images/binary"):
    os.makedirs("Euler_Images/binary")

imageio.imsave('Euler_Images/binary/max_16_reduced.png', max_16_reduced.astype('uint8') * 255)# save image

def AND_Euler_wth_SI (SI, img):
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if (SI[x,y] == 1) and (img[x,y] == 1):
                img[x,y] = 1
            else:
                img[x,y] = 0
                
    return img

AND_Euler = AND_Euler_wth_SI(SI_img, max_16_reduced)

imageio.imsave('Euler_Images/binary/AND_Euler.png', AND_Euler.astype('uint8') * 255)# save image
